### Preamble:

In [20]:
%matplotlib inline
import pandas as pd, numpy as np, os
from pandas import DataFrame, Series, HDFStore
from datetime import datetime
pd.set_option('display.max_rows', 50)

In [14]:
#Depending on which computer I'm working on (Mac/Windows)... change  directory to the correct folder
from sys import platform as _platform
if _platform == 'darwin':
    os.chdir(os.getcwd()+'/Master Download/Table 1 - Stock Transactions')
elif _platform == 'win64' or _platform =='win32':
    os.chdir(os.getcwd()+'\\Master Download\\Table 1 - Stock Transactions')

### Define which columns we won't need:

In [3]:
drop_cols=['FDATE', 'CDATE', 'SEQNUM','OWNER','SHARES','SIGDATE','ACQDISP_AR','SHARESHELD',
           'ADDRESS1','ADDRESS2','CITY','COUNTRY','STATE','SECID', 'OWNER','MAINTDATE',
           'CUSIP2','PHONE','ZIPCODE','DCN','FORMTYPE','TRANCODE_AR','SECTOR',
            'SECTOR','CNUM','ROLECODE2','ROLECODE3','SECDATE',
            'TRANDATE_AR','TPRICE','SECTITLE','TPRICE_AR','ROLECODE4']

keep_cols= ['CNAME','CUSIP6','TICKER','PERSONID','ROLECODE1','ACQDISP','TRANDATE','TRANCODE','OWNERSHIP',
            'OPTIONSELL','CLEANSE','AMEND','INDUSTRY','SHARESHELD_ADJ','TPRICE_ADJ','SHARES_ADJ']

### Define a function to tidy up data.  Keep valid (non-null) acquisition/disposition data given by ACQDISP, and dropping trash data defined by Thomson-Reuters given by CLEANSE 

In [4]:
def tidy_data(df):
    #df=df.drop(drop_cols,axis=1)
    #df=df.loc[:,keep_cols] #Keep the columns specified in above cell
    #df['TRANDATE']=pd.to_datetime(df['TRANDATE'])

    #TRANCODE == ["P","S"] and
#    df= df.query('TRANCODE == ["P","S"] and ACQDISP in ["A","D"] and CLEANSE!="A"')
    df= df.query('TRANCODE == ["P"] and ACQDISP in ["A","D"] and CLEANSE!="A"')
    
    #reorder the columns
    df=df.reset_index() #reset the index
    df=df.drop('index',axis=1)
    return df.sort('CUSIP6')

## Load the data:

###Run this block if we want to load all data from scratch again
all_data=pd.read_csv('2003.csv',engine='c',parse_dates=['TRANDATE'], low_memory=False,usecols=keep_cols)
for iter in xrange(4,13+1):
    if iter <10: #if year is one digit, stick a zero on before
        year= '200%i' %iter
        df=pd.read_csv(year+'.csv',engine='c',parse_dates=['TRANDATE'], low_memory=False,usecols=keep_cols)
        all_data=pd.concat([all_data,df],axis=0,ignore_index=True); #concatenate all data
    else: #if year is two digits, don't stick a zero on
        year='20%i' %iter
        df=pd.read_csv(year+'.csv',engine='c' ,parse_dates=['TRANDATE'],low_memory=False,usecols=keep_cols)
        all_data=pd.concat([all_data,df],axis=0,ignore_index=True); #concatenate all data

###Run this block to save data in efficient hdf5 format
raw=pd.HDFStore('raw_data.h5')
raw['data']=all_data
raw.close()

In [22]:
#Run this block to load from concatenated dataset
all_data = pd.read_hdf('raw_data.h5','data')

### Clean the data using the tidy_data():

In [23]:
cleaned=tidy_data(all_data) 

In [24]:
year=pd.DatetimeIndex(cleaned.TRANDATE).year
cleaned['YEAR']=year

###Absolute number of purchases made by ALL insiders in each year.

In [28]:
PurchasesAll=cleaned.groupby(['YEAR'])
PurchasesAll.size()

YEAR
2003     42292
2004     43243
2005     52563
2006     51626
2007     72295
2008    109694
2009     48570
2010     25854
2011     38780
2012     32987
2013      5718
dtype: int64

###Absolute number of purchases made by CEOs in each year.

In [29]:
cleanedCEO = cleaned.query('ROLECODE1==["CEO"]')
cleanedCEO = cleanedCEO.sort('YEAR')
PurchasesCEO = cleanedCEO.groupby(['YEAR'])
PurchasesCEO.size()

YEAR
2003     5293
2004     4557
2005     5952
2006     7212
2007    10017
2008    14628
2009     8022
2010     3269
2011     6324
2012     5862
2013     1118
dtype: int64

### Fraction of absolute trades by CEOs only

In [30]:
PurchasesCEO.size()/PurchasesAll.size()

YEAR
2003    0.125154
2004    0.105381
2005    0.113236
2006    0.139697
2007    0.138557
2008    0.133353
2009    0.165164
2010    0.126441
2011    0.163074
2012    0.177706
2013    0.195523
dtype: float64